## Scraping video data using YouTube API

#### Set-up

Run the following instructions on your terminal:
``` bash
pip install google-api-python-client
pip install --upgrade google-api-python-client
```

In [1]:
from googleapiclient.discovery import build
from IPython.display import JSON
from pprint import pprint
import pandas as pd
import json

In [2]:
credentials_file_path = "../credentials.json"

# open the file and load the data into a variable
with open(credentials_file_path, "r") as f:
    credentials = json.load(f)

In [3]:
youtube = build('youtube', 'v3', developerKey=credentials['youtube_api_key'])

In [4]:
# Create the request object
data = []

search_response = youtube.search().list(
    part="snippet",
    maxResults=100,
    q="official music video",
    regionCode="US",
    type="video",
    videoCategoryId="10",
    fields="items(id/videoId,snippet(channelId,channelTitle,description,title)),nextPageToken,pageInfo,prevPageToken,regionCode"
)

# Execute the request and get the response
response = search_response.execute()

# Extract video IDs from the response
video_ids = [item['id']['videoId'] for item in response['items']]

for item in response['items']:
    video_id = item['id']['videoId']
    title = item['snippet']['title']
    channel_id = item['snippet']['channelId']
    channel_title = item['snippet']['channelTitle']
    description = item['snippet']['description']

    data.append({
        'video_id': video_id,
        'title': title,
        'channel_id': channel_id,
        'channel_title': channel_title,
        'description': description
    })

df = pd.DataFrame(data)

In [5]:
### getting video statistics (likes, views, comments etc.)

video_stats = []
video_requests = youtube.videos().list(
    part = "snippet, statistics, contentDetails",
    id = video_ids
)

video_responses = video_requests.execute()

In [10]:
type(video_responses)

dict

In [6]:
like_count = video_responses['items'][0]['statistics']["likeCount"]
view_count = video_responses['items'][0]['statistics']["viewCount"]
like_count, view_count

('949878', '65065267')